# Analyzing CIA Fact Book Using SQL

# Introduction

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

1. population — the global population.
2. population_growth — the annual population growth rate, as a percentage.
3. area — the total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to analyze data from this database.

In [39]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# Overview of the Data

In [40]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics

In [41]:
%%sql
SELECT MIN(population) AS Minimum_Population, 
MAX(population) AS Maximum_Population, 
MIN(population_growth) AS Minimum_Population_Growth, 
MAX(population_growth) AS Maximum_Population_Growth
  FROM facts;

 * sqlite:///factbook.db
Done.


Minimum_Population,Maximum_Population,Minimum_Population_Growth,Maximum_Population_Growth
0,7256490011,0.0,4.02


# Exploring Outliers

In [42]:
%%sql
SELECT name AS Minimum_Population_Country
  FROM facts
 WHERE population = (SELECT MIN(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


Minimum_Population_Country
Antarctica


In [43]:
%%sql
SELECT name AS Maximum_Population_Country
  FROM facts
 WHERE population = (SELECT MAX(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


Maximum_Population_Country
World


We can see that Antartica is listed as the country with the least population. Although Antartica is not a country, this is how it is listed in the CIA Fact Book. We will keep this row. However, the entire world is also listed as a country and it contains all 7.2 Billion inhabitants on Earth. We will continue our analysis without this row.

# Exploring Average Population and Area

In [44]:
%%sql
SELECT MIN(population) AS Minimum_Population, 
MAX(population) AS Maximum_Population, 
MIN(population_growth) AS Minimum_Population_Growth, 
MAX(population_growth) AS Maximum_Population_Growth
  FROM facts
 WHERE population < (SELECT MAX(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


Minimum_Population,Maximum_Population,Minimum_Population_Growth,Maximum_Population_Growth
0,1367485388,0.0,4.02


In [45]:
%%sql
SELECT ROUND(AVG(population), 2) AS Average_Population, 
ROUND(AVG(area), 2) AS Average_Area
  FROM facts
 WHERE population < (SELECT MAX(population)
                      FROM facts
                    );

 * sqlite:///factbook.db
Done.


Average_Population,Average_Area
32242666.57,582949.85


# Finding Densely Populated Areas

We will now find densely populated countries by identifying countries that have above average population and below average area.

In [46]:
%%sql
SELECT name AS Dense_Countries
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                     WHERE name <> 'World'
                    )
 AND area < (SELECT AVG(area)
              FROM facts
             WHERE name <> 'World'
            );

 * sqlite:///factbook.db
Done.


Dense_Countries
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


# Finding Most Populated Country

In [47]:
%%sql 
SELECT name, MAX(population)
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


name,MAX(population)
China,1367485388


We can see that China has the most population.

# Finding Highest Growth Rate

In [48]:
%%sql 
SELECT name, MAX(population_growth)
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


name,MAX(population_growth)
South Sudan,4.02


We can see that South Sudan has the highest population growth.

In [49]:
%%sql 
SELECT name, MAX(birth_rate + migration_rate - death_rate) AS EstimatedGrowth
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


name,EstimatedGrowth
South Sudan,40.199999999999996


We can see that South Sudan has the highest growth rate and also will add the most people to their populations next year. This was calculated by adding the birth and migration rate, and then subtracting the death rate.

# Higher Death Rate than Birth Rate

In [50]:
%%sql
SELECT name
  FROM facts
 WHERE name <> 'World'
  AND death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


We can see that many European countries have a higher death rate than birth rate.

# Water to Land Ratio

Now, we will calculate which five countries have the highes ratios of water to land. In other words, which countries have more water than land.

In [51]:
%%sql
SELECT name, CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT) AS Water_to_Land_Ratio
  FROM facts
 WHERE name <> 'World'
ORDER BY Water_to_Land_Ratio DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,Water_to_Land_Ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623


We can see that these countries are all island nations.

# Highest Population to Area Ratio

Now, we will see which countries have the highest ratio of population to area and compare it to the result of the densely populated query.

In [52]:
%%sql 
SELECT name, CAST(population AS FLOAT) / CAST(area AS FLOAT) AS Population_to_Area_Ratio
  FROM facts
 WHERE name <> 'World'
ORDER BY Population_to_Area_Ratio DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,Population_to_Area_Ratio
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444


We can see that the list of top five countries with the highest population to area ratio does not compare to the densely populated query from earlier. Based on our common knowledge, we can see that these countries above are small nations, with likely below average population and below average area. Our query from earlier, had a WHERE clause that listed population as above average. Since these countries have smaller populations, they did not appear in the query.